In [2]:
import pandas as pd
import numpy as np
import time
from datetime import datetime

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity

import re
import string
from bs4 import BeautifulSoup
import json

import nltk
from nltk.corpus import stopwords
import pymorphy2

from tqdm import tqdm_notebook
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
time_format = "%H:%M:%S"
pd.set_option('display.max_colwidth', -1)

In [4]:
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()
splitting_pattern = "[" + string.punctuation + string.whitespace + "]"

In [5]:
def get_clean_text(text, remove_tags=True, can_be_json=True):
    if can_be_json:
        try:
            text = get_text_from_json(text)
        except ValueError:
            pass

    if remove_tags:
        text = BeautifulSoup(text, "lxml").get_text()
    
    return " ".join(get_normalized_words(text))


def get_text_from_json(text):
    text_list = []
    
    jso = json.loads(text)
    for o in jso:
        try:
            text_list.append(o['value']['blocks'][0]['text'])
        except (KeyError, TypeError):
            pass
    
    return " ".join(text_list)


def get_normalized_words(text):
    normalized_words = []
    
    for word in re.split(splitting_pattern, text):
        norm_word = morph.parse(word)[0].normal_form
        if norm_word != "" and norm_word not in stop_words:
            normalized_words.append(norm_word)
    
    return normalized_words

In [8]:
print("{}: reading train data...".format(datetime.now().strftime(time_format)))
start = time.time()

train_data = pd.read_csv('./train.csv', 
                          names=["post_id", "published_at", "title", "subtitle", "content"], 
                          header=0,
                          na_filter=False)

elapsed_time = time.time() - start
print("Elapsed time: {:.4f} sec.".format(elapsed_time))

20:16:21: reading train data...
Elapsed time: 9.8749 sec.


Очистим текст от тегов и приведем слова к нормальной форме. Т.к. вычисления занимают около 25ти часов, сохраним результаты в файл.

In [29]:
#print("{}: cleaning train data...".format(datetime.now().strftime(time_format)))
#start = time.time()

#for it in tqdm_notebook(train_data.index):
#     title = clean_train_data.loc[it].title
#     subtitle = clean_train_data.loc[it].subtitle
#     content = train_data.loc[it].content
#    
#     train_data.at[it, "clean_title"] = get_clean_text(title)
#     train_data.at[it, "clean_subtitle"] = get_clean_text(subtitle)
#     train_data.at[it, "clean_content"] = get_clean_text(content)

#train_data[['post_id', 'clean_title', 'clean_subtitle', 'clean_content']].to_csv("cleaned_train_data.csv", index = False)

#elapsed_time = time.time() - start
#print("Elapsed time: {:.4f} sec.".format(elapsed_time))

In [10]:
print("{}: reading cleaned train data...".format(datetime.now().strftime(time_format)))
start = time.time()

train_data = pd.read_csv('./cleaned_train_data.csv',
                         names=['post_id', 'clean_title', 'clean_subtitle', 'clean_content'], 
                         header=0,
                         na_filter=False)

elapsed_time = time.time() - start
print("Elapsed time: {:.4f} sec.".format(elapsed_time))

07:54:22: reading cleaned train data...
Elapsed time: 8.5848 sec.


In [11]:
train_data

,post_id,clean_title,clean_subtitle,clean_content
0,1,полиция стамбул собираться штурмовать ночной клуб произойти нападение,согласно последний данные здание мочь находиться минимум нападать,полиция стамбул собираться штурмовать ночной клуб произойти нападение согласно последний данные минимум злоумышленник находиться здание раненый разный данные 40 60 человек жертва стать менее четыре человек среди погибший — полицейский время празднеств ть клуб рейн находиться 800 человек неизвестный произвести множество выстрел сообщаться взорвать гранат
1,2,расстрел посетитель ночное клуб стамбул попасть видео,ранее сообщаться полицейский собираться штурмовать здание развлекательный заведение,опубликовать видео злоумышленник расстреливать посетитель ночное клуб рейн стамбул согласно последний данные нападение погибнуть менее четырёх человек среди который полицейский здание клуб момент нападение находиться 800 человек разный данные раненый 40 60 человек правоохранитель собираться штурмовать рейн клуб напали от человек сообщаться минимум злоумышленник мочь находиться клуб
2,3,губернатор стамбул нападение ночной клуб погибнуть 35 человек,ранее сообщаться 40 60 человек получить ранение различный степень тяжесть,губернатор стамбула васип шахин заявить нападение ночной клуб рейн погибнуть 35 человек предварительный информация раненый 40 60 человек сообщать очевидец клуб располагаться берег многие посетитель прыгать вода пытаться спастись нападать экстренный служба работать место человек спасать вода ход развитие событие следить наш текстовый трансляция
3,4,губернатор стамбул назвать террористический атака нападение клуб рейн,согласно последний данные погибнуть минимум 35 человек раненый около 40 посетитель,губернатор стамбул васип шахин назвать террористический атака нападение ночной клуб рейн результат который слово погибнуть минимум 35 человек разный данные раненый 40 60 нападать человек предварительный информация террорист вооружённый ак 47 гранат прогреметь сообщаться менее взрыв момент нападение клуб находиться 800 человек многие паника прыгать босфор экстренный служба спасать пострадавший вода
4,5,очевидец ночной клуб стамбул проходить беспрепятственно,ранее губернатор стамбул васип шахин назвать произойти террористический атака,посетитель стамбульский клуб рейн который совершено нападение заявить проходить беспрепятственно бронирование столик также требоваться напомнить губернатор стамбул васип шахин назвать произойти террористический атака последний данные нападать сообщаться террорист сей пора мочь находиться клуб согласно последний данные погибнуть минимум 35 человек ранение получить разный данные 40—60 человек
5,6,турецкий клуб который атаковать террорист серьёзный фейс контроль,,клуб рейн который сегодня совершить вооружённый нападение иметь достаточно высокий цена серьёзный охрана вход заведение жёсткий фейс контроль однако вход туда разрешить предварительный резервирование столик губернатор стамбул васип шахин назвать нападение клуб теракт также отметить данные результат атака погибнуть менее 35 человек
6,7,опубликовать видео клуб рейн несколько часы нападение,ранее сообщаться результат атака погибнуть 35 человек менее 40 получить ранение,опубликовать видео стамбульский клуб рейн несколько часы нападение заметно ролик здание находиться множество человек праздновать жечь бенгальский огонь танцевать громкий музыка губернатор стамбул васип шахин назвать террористический атака нападение рейн согласно последний данные злоумышленник вооружённый гранат ак 47 произвести взрыв открыть беспорядочный огонь посетитель напугать человек прыгать окно здание согласно последний данные результат нападение погибнуть минимум 35 человек раненый менее 40 клуб мочь находиться 800 посетитель
7,8,турецкий сми запретить писать теракт стамбул,,официальный власть турция экстренно издать новый указ местный сми который запрещать публиковать любовать информация произойти теракт стамбульский ночное клуб предварительный данные погибнуть 35 человек ранее сообщаться 

In [12]:
print("{}: vectorizing data...".format(datetime.now().strftime(time_format)))
start = time.time()

n_features = 18000
count_vectorizer = CountVectorizer(max_features=n_features, stop_words=stop_words)
tf = count_vectorizer.fit_transform(train_data["clean_content"])
feature_names = count_vectorizer.get_feature_names()

elapsed_time = time.time() - start
print("Elapsed time: {:.4f} sec.".format(elapsed_time))

07:54:44: vectorizing data...
Elapsed time: 34.9972 sec.


In [15]:
print("{}: fitting LDA model...".format(datetime.now().strftime(time_format)))
start = time.time()

n_topics = 15
lda = LatentDirichletAllocation(n_components = 15, 
                                max_iter = 10, 
                                learning_method = 'online', 
                                learning_offset = 50.0,
                                random_state = 0
                               ).fit(tf)

elapsed_time = time.time() - start
print("Elapsed time: {:.4f} sec.".format(elapsed_time))

08:00:21: fitting LDA model...
Elapsed time: 1259.0329 sec.


In [16]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [17]:
n_top_words = 7
display_topics(lda, feature_names, n_top_words)

Topic 0:
мужчина женщина её который девушка свой это
Topic 1:
год свой ребёнок её который стать летний
Topic 2:
место человек произойти сотрудник автомобиль лайф водитель
Topic 3:
президент сша это россия заявить трамп страна
Topic 4:
самолёт сообщать метр пассажир сообщить служба аэропорт
Topic 5:
компания который пользователь новый это смартфон также
Topic 6:
суд год дело задержать убийство который бывший
Topic 7:
игра год команда фильм матч который это
Topic 8:
украина украинский это страна военный который киев
Topic 9:
год тысяча миллион рубль это доллар компания
Topic 10:
человек сообщать это организация который город данные
Topic 11:
год это мочь который ракета весь время
Topic 12:
весь это который свой человек самый мочь
Topic 13:
это который россия также рф год российский
Topic 14:
это который человек мочь врач учёный ребёнок


Calculate similarity score on specified documents:

In [22]:
print("{}: reading sample file...".format(datetime.now().strftime(time_format)))
start = time.time()

sample_data = pd.read_csv('sample_submission.csv', 
                         names=["id", "similarity"], 
                         header=0)


08:38:09: reading sample file...


In [23]:
def get_similarity(pair_id):
    id_1, id_2 = pair_id.split("_")
    
    text_1 = train_data['clean_content'].iloc[int(id_1) - 1]
    text_2 = train_data['clean_content'].iloc[int(id_2) - 1]
    
    lda_vector_1 = lda.transform(count_vectorizer.transform([text_1])) 
    lda_vector_2 = lda.transform(count_vectorizer.transform([text_2]))

    return cosine_similarity(lda_vector_1, lda_vector_2)[0][0]

In [25]:
sample_data["similarity"] = sample_data['id'].apply(get_similarity)

In [26]:
sample_data.sort_values(by=['similarity'], ascending=False).head()

,id,similarity
1366,13127_17987,1.0
8203,3540_34092,1.0
2604,34264_8303,1.0
3519,5870_36210,1.0
9653,7990_42367,1.0


In [27]:
sample_data

,id,similarity
0,38277_31392,0.925374
1,79621_65626,0.998472
2,61134_17553,0.021316
3,56944_64787,0.275027
4,48365_7040,0.959394
5,12762_38868,0.029200
6,9185_33260,0.006151
7,40675_101417,0.358774
8,36897_34524,0.033550
9,39233_45755,0.242385


In [28]:
file_name = "out.csv".format(n_features, n_topics)
sample_data.to_csv(file_name, index = False)